# Automatically fetching app analytics data from AppStoreConnect

## Modules import

In [ ]:
import json
import requests as re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

## Fetching the bot configuration parameters and the credentials to login to AppStoreConnect

In [ ]:
# Opening JSON file
with open('config.json', 'r') as openfile:
    # Reading from json file
    config_dict = json.load(openfile)

In [ ]:
profile_path = config_dict["profile_path"]
profile_directory = config_dict["profile_directory"]

email_address_login = config_dict["email_address_login"]
password_login = config_dict["password_login"]

## Generating and recovering the necessary cookie parameter

### Recovering the Chrome profile that verified apple's double authentication by sms 
The double authentication needs to be verified once every three weeks I believe

In [ ]:
service = Service()
chrome_options = Options()
chrome_options.add_argument(f"--user-data-dir={profile_path}")
chrome_options.add_argument(f"--profile-directory={profile_directory}")

### Bot initialization and cookie parameter fetching

In [ ]:
myacinfo_cookie = ""

'''
Bot initialization on the previously defined Chrome profile
'''
driver = webdriver.Chrome(service=service, options=chrome_options)
driver.maximize_window()

'''
Bot connection to the AppStoreConnect
'''
driver.get("https://appstoreconnect.apple.com/login")
driver.switch_to.default_content()
wait = WebDriverWait(driver, 10)

iframe = wait.until(EC.presence_of_element_located((By.NAME, "aid-auth-widget")))
driver.switch_to.frame(iframe)

'''
Website login
'''
email_address = wait.until(EC.presence_of_element_located((By.ID, "account_name_text_field")))
wait.until(EC.element_to_be_clickable(email_address)).send_keys(email_address_login)
time.sleep(0.1)
email_address.send_keys(Keys.RETURN)

password = wait.until(EC.presence_of_element_located((By.ID, "password_text_field")))
wait.until(EC.element_to_be_clickable(password)).send_keys(password_login)
time.sleep(0.1)
password.send_keys(Keys.RETURN)

'''
Generating the cookie parameter by navigating to the analyses tab
'''
wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="main-nav"]/div[1]/div[2]/a'))).click()

'''
Recovering the cookie parameter
'''
cookies = driver.get_cookies()
found = False
index = 0
while (not found) and (index < len(cookies)):
    if cookies[index]["name"] == "myacinfo":
        myacinfo_cookie = cookies[index]["value"]
        found = True
    else:
        index += 1
if found:
    print("SUCCESS: Successfully recovered cookie")
else:
    print("FAILURE: Cookie not found")
driver.close()
driver.quit()

## Connecting to the internal API and recovering the desired app analysis data

### Initializing the connection parameters and the data measures we wish to recover

In [ ]:
# Fetching the desired data, which in this case are the app's total downloads, first downloads and redownloads

url = "https://appstoreconnect.apple.com/analytics/api/v1/data/time-series"
cookie_headers= config_dict["cookie_header"] + f" myacinfo={myacinfo_cookie}"
adam_id = config_dict["adam_id"]

measures = ["totalDownloads", "redownloads", "units"]
freq = "day"
start_time = "2023-01-01T00:00:00Z"
end_time = "2023-02-01T00:00:00Z"

headers = headers = {
  'Host': 'appstoreconnect.apple.com',
  'X-Requested-By': 'dev.apple.com',
  'Cookie': cookie_headers,
  'Content-Type': 'application/json'
}

### Data fetching

In [ ]:
results = []
for measure in measures:
  payload = json.dumps({"adamId":[adam_id],"measures":[measure],"frequency":freq,"startTime":start_time,"endTime":end_time})
  results.append(re.request("POST", url, headers=headers, data=payload, allow_redirects=False).text)
  print(re.request("POST", url, headers=headers, data=payload, allow_redirects=False).text)

### Formatting the recovered data into a pandas dataframe

In [ ]:
totalDownloads = pd.json_normalize(json.loads(results[0])["results"][0]["data"])
firstDownloads = pd.json_normalize(json.loads(results[2])["results"][0]["data"]).rename(columns={"units": "firstDownloads"})
reDownloads = pd.json_normalize(json.loads(results[1])["results"][0]["data"])
df = totalDownloads.join(firstDownloads.set_index("date"), on="date").join(reDownloads.set_index("date"), on="date")
df.head()